## Using the XBRL Rule and Query Specification (XULE) to normalize an SEC Report  
This interactive environment copies the [free, open-source XULE plugin](https://github.com/xbrlus/xule/) to a working Python environment with [Arelle](https://pypi.org/project/arelle-release/) plus [additional packages](https://raw.githubusercontent.com/xbrlus/xule/jupyter/environment.yml). Click the run button to execute the install steps. **This step only needs to be completed once for the notebook session** unless different versions of Arelle or XULE are required. Click the `Show code` link to expose the script for establishing Arelle, XULE and necessary plugins.

In [ ]:
# @title
import os, shutil, sys, site, platform
print('Please wait while Arelle, XULE and some helper packages are installed. \nA XULE version message appears below when the environment is ready.')

# In this example, Arelle and aniso are required to use XULE - get Arelle release details from GitHub (https://github.com/Arelle/arelle/releases).
# Use %pip -q install git+https://git@github.com/Arelle/arelle.git@master to use Arelle's development release
%pip -q install Arelle-release==2.30.11
%pip -q install aniso8601==9.0.1

# 1) locate Arelle's plugin directory (do not modify this location); remove temp and xuledir if they exist
plugindir = site.getsitepackages()[0] + '/arelle/plugin/'
xuledir = plugindir + 'xule/'
xodeldir = plugindir + 'xodel/'
serializerdir = plugindir + 'serializer/'
SimpleXBRLModeldir = plugindir + 'SimpleXBRLModel/'
temp = plugindir + 'temp/'
tempxule = temp + 'plugin/xule/'
tempxodel = temp + 'plugin/xodel/'
tempserializer = temp + 'plugin/serializer/'
tempSimpleXBRLModel = temp + 'plugin/SimpleXBRLModel/'
if os.path.exists(temp):
        if os.path.isdir(temp):
            os.remove(plugindir + 'semanticHash.py')
            shutil.rmtree(xuledir)
            shutil.rmtree(xodeldir)
            shutil.rmtree(serializerdir)
            shutil.rmtree(SimpleXBRLModeldir)
            shutil.rmtree(temp)
else: ''
os.chdir(plugindir)

# 2) copy a XULE release from GitHub (https://github.com/xbrlus/xule/releases) to plugin directory
!git clone --quiet --depth=1 --branch 30041 --single-branch https://github.com/xbrlus/xule.git temp &> /dev/null
shutil.move(temp + 'plugin/semanticHash.py', plugindir)
shutil.move(tempxule, xuledir)
shutil.move(tempxodel, xodeldir)
shutil.move(tempserializer, serializerdir)
shutil.move(tempSimpleXBRLModel, SimpleXBRLModeldir)

# 3) confirm XULE (change -v to -h and re-run to see help contents for Arelle and XULE)
!arelleCmdLine --plugins 'xule' -v

### 2. Run XULE against an XBRL report

The Python command below invokes Arelle and runs an SEC report created with the 2024 US GAAP Taxonomy against a compiled XULE .zip file. The underlying code uses Xodel - a XULE module that builds taxonomies and can create reports from structured data sources - to transform the existing report into an XML instance document (report) based on a standardized version of US GAAP (entry point: [https://taxonomies.xbrl.us/standardized-statements/xusss.xsd](https://taxonomies.xbrl.us/standardized-statements/xusss.xsd))

|    code arguments |    |
|    --------    |    -------    |
|    --plugins 'xodel\|\transforms/SEC\|\validate/EFM\|\inlineXbrlDocumentSet'  |    --httpUserAgent='XULE-Arelle (xbrl.us; support@xbrl.us)'    
|    -f https://www.sec.gov/Archives/edgar/data/700764/000107997324001419/0001079973-24-001419-xbrl.zip   |    --xule-rule-set https://raw.githubusercontent.com/xbrlus/xule/jupyter/2024-ugt-norm.zip  |    
|    --xule-arg TAXONOMY_LOC=https://taxonomies.xbrl.us/standardized-statements/xusss.xsd  |    --xule-arg FILING_URL=http://www.sec.gov/Archives/edgar/data/700764/000107997324001419/vyey_10q-033124.htm  |   
|    --xince-file-type xml |     --xodel-location /content/ |    
|    --xule-arg InstanceName=standardized      |     --xule-arg format=xml |
|    --xule-arg FILING_DATE=2024-10-17 |     --xule-arg SIC_CODE=1311 |
|    --xule-arg FILING_ACCESSION=0001079973-24-001419 |    |  

This argument tells the process which sections of the report should be created in the instance:  
 --xule-run-only createInstance,make_cf_fact_values,make_bs_fact_values,make_FA,make_OA,make_AP,make_AE,make_LTDC,make_OLC,  
 make_UCL,make_LTDNC,make_OLNC,make_LE,make_is_fact_values,make_standard_data,get_dei_data,get_filing_data,get_bus_desc,  
 get_tax_data,get_royalty_data,get_marketing,make_STDC,get_trading_symbol  

 !arelleCmdLine --plugins 'xodel|transforms/SEC|validate/EFM|inlineXbrlDocumentSet' --httpUserAgent='XULE-Arelle (xbrl.us; support@xbrl.us)' -f https://www.sec.gov/Archives/edgar/data/700764/000107997324001419/0001079973-24-001419-xbrl.zip --xule-rule-set https://raw.githubusercontent.com/xbrlus/xule/jupyter/2024-ugt-norm.zip --xule-arg TAXONOMY_LOC=https://taxonomies.xbrl.us/standardized-statements/xusss.xsd --xule-arg FILING_URL=http://www.sec.gov/Archives/edgar/data/700764/000107997324001419/vyey_10q-033124.htm --xince-file-type xml --xodel-location /content/ --xule-arg InstanceName=standardized --xule-arg format=xml --xule-arg FILING_DATE=2024-10-17 --xule-arg SIC_CODE=1311 --xule-arg FILING_ACCESSION=0001079973-24-001419

In [ ]:
SEC_URL = input('Enter an SEC financial report based on the 2024 US GAAP Taxonomy: ')
!arelleCmdLine --plugins 'xodel|transforms/SEC|validate/EFM|inlineXbrlDocumentSet' --httpUserAgent='XULE-Arelle (xbrl.us; support@xbrl.us)' -f SEC_URL --xule-rule-set https://raw.githubusercontent.com/xbrlus/xule/jupyter/2024-ugt-norm.zip --xule-arg TAXONOMY_LOC=https://taxonomies.xbrl.us/standardized-statements/xusss.xsd --xule-arg FILING_URL=http://www.sec.gov/Archives/edgar/data/700764/000107997324001419/vyey_10q-033124.htm --xince-file-type xml --xodel-location /content/ --xule-arg InstanceName=standardized --xule-arg format=xml --xule-arg FILING_DATE=2024-10-17 --xule-arg SIC_CODE=1311 --xule-arg FILING_ACCESSION=0001079973-24-001419


The commands in the cell above:
1. uses the XULE plugin to compile the XULE file from step 1 into a .zip file (XULE must be compiled before it will run)
1. copies the source .xule and compiled .zip files to the content folder in the environment (click the folder at left to download), and
1. uses XULE and other plugins to run the .zip against an XBRL report listed in the command (the SEC requires a User Agent to access reports programmatically), create a file according to the XULE expression (.xlsx or .json) and generate a log.xml of the process.

The XULE expression and Arelle commands define `/content/` as the location for files - XULE, .zip, .xlsx and .xml - so these outputs will appear in the folder at left (default Colab environment) after processing completes.

If the `$schema` constant in the expression is updated for a 'base' taxonomy (ie. FASB, FERC, IFRS), a report - also known as an instance document - using the same base taxonomy must be used with the last command in the sequence.

(NB: run the first code cell in this notebook with the -h switch instead of -v to get configuration paramters for the command).
